In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
from torch.utils.tensorboard import SummaryWriter # pip install --force-reinstall charset-normalizer==3.1.0
from torch import optim
from data_utils import *
from model_utils import *

2024-05-31 13:34:21.080352: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-31 13:34:21.083681: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-31 13:34:21.131349: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-31 13:34:22.010603: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# print(device)

n_cudas = torch.cuda.device_count()
print(f"gpu count: {n_cudas}")
for i in range(n_cudas):
    print(torch.cuda.get_device_name(i))

gpu count: 0


# Prepare Dataset

In [3]:
# create raw dataset
x, y = GenerateRandomDataset.generate_slr_dataset_v2(
    true_bias=3.0, 
    true_weight=4.0, 
    sample_size=1000
)

In [4]:
# into Pytorch's tensor
x_tensor = torch.as_tensor(x).float().to(device)
y_tensor = torch.as_tensor(y).float().to(device)

In [5]:
# builds datasets
dataset = CustomDataset(x_tensor, y_tensor)

In [6]:
# performs the split
ratio = .8
n_total = len(dataset)
n_train = int(n_total * ratio)
n_val = n_total - n_train
train_data, val_data = random_split(dataset, [n_train, n_val])

In [7]:
# builds dataloaders
train_loader = DataLoader(dataset=train_data, batch_size=16, shuffle=True)
val_loader = DataLoader(dataset=val_data, batch_size=16, shuffle=True)

In [8]:
# retrieve mini-batches
# next(iter(train_loader))

# Define Model

In [9]:
# define model
class MyLinearRegressionNested(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(in_features=1, out_features=1)
    
    def forward(self, x):
        y_hat = self.linear(x)
        return y_hat

In [10]:
# create model instance
reg_model = MyLinearRegressionNested().to(device)

# Training Loop

## Mini-batch Gradient Descent

In [11]:
num_epochs = 20
lr = 0.1
optimizer = optim.SGD(params=reg_model.parameters(), lr=lr)
loss_fn = nn.MSELoss(reduction='mean')

perform_train_step = make_train_step_fn(model=reg_model, loss_func=loss_fn, optimizer=optimizer)
perform_val_step = make_val_step_fn(model=reg_model, loss_func=loss_fn)

In [12]:
losses = []
val_losses = []
for i in range(num_epochs):
    # train
    loss = mini_batch(device=device, data_loader=train_loader, step_fn=perform_train_step)
    losses.append(loss)
    
    # validation
    with torch.no_grad():
        val_loss = mini_batch(device=device, data_loader=val_loader, step_fn=perform_val_step)
        val_losses.append(val_loss)
    
    print(f"Epoch: {i}, Training Loss: {loss:0.5f}, Validation Loss: {val_loss:0.5f}")

Epoch: 0, Training Loss: 1.21706, Validation Loss: 0.03525
Epoch: 1, Training Loss: 0.02363, Validation Loss: 0.01420
Epoch: 2, Training Loss: 0.01366, Validation Loss: 0.01228
Epoch: 3, Training Loss: 0.01116, Validation Loss: 0.01084
Epoch: 4, Training Loss: 0.01057, Validation Loss: 0.01083
Epoch: 5, Training Loss: 0.01031, Validation Loss: 0.01063
Epoch: 6, Training Loss: 0.01031, Validation Loss: 0.01018
Epoch: 7, Training Loss: 0.01029, Validation Loss: 0.00986
Epoch: 8, Training Loss: 0.01026, Validation Loss: 0.00996
Epoch: 9, Training Loss: 0.01051, Validation Loss: 0.01124
Epoch: 10, Training Loss: 0.01024, Validation Loss: 0.01043
Epoch: 11, Training Loss: 0.01019, Validation Loss: 0.01075
Epoch: 12, Training Loss: 0.01025, Validation Loss: 0.01104
Epoch: 13, Training Loss: 0.01016, Validation Loss: 0.01104
Epoch: 14, Training Loss: 0.01024, Validation Loss: 0.01038
Epoch: 15, Training Loss: 0.01029, Validation Loss: 0.00985
Epoch: 16, Training Loss: 0.01030, Validation Loss

In [13]:
# Checks model's parameters
print(reg_model.state_dict())

OrderedDict({'linear.weight': tensor([[3.9936]]), 'linear.bias': tensor([2.9960])})


# Saving Models

In [20]:
checkpoint = {
    'epoch': num_epochs,
    'model_state_dict': reg_model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': losses,
    'val_loss': val_losses
}

# save checkpoint
torch.save(checkpoint, 'reg_model_checkpoint_31052024.pth')

# Loading Models

In [24]:
# create a new model instance 
# as if there is no model in memory

new_model = MyLinearRegressionNested().to(device)
new_optimizer = optim.SGD(params=new_model.parameters(), lr=lr)

In [22]:
new_model.state_dict()

OrderedDict([('linear.weight', tensor([[-0.8549]])),
             ('linear.bias', tensor([-0.2070]))])

In [25]:
# load model components
checkpoint_to_load = torch.load('reg_model_checkpoint_31052024.pth')

new_model.load_state_dict(checkpoint_to_load['model_state_dict'])
new_optimizer.load_state_dict(checkpoint_to_load['optimizer_state_dict'])

saved_epoch = checkpoint['epoch']
saved_losses = checkpoint['loss']
saved_val_losses = checkpoint['val_loss']

In [26]:
new_model.train()

MyLinearRegressionNested(
  (linear): Linear(in_features=1, out_features=1, bias=True)
)

In [27]:
print(new_model.state_dict())

OrderedDict({'linear.weight': tensor([[3.9936]]), 'linear.bias': tensor([2.9960])})


# Prediction

In [29]:
new_inputs = torch.tensor([[0.20], [0.34], [0.57]])

# set the model to eval mode
new_model.eval()

# compute predictions
new_model(new_inputs.to(device))

tensor([[3.7947],
        [4.3538],
        [5.2723]], grad_fn=<AddmmBackward0>)